In [8]:
# For use with model vs other targets from Tim's code
# LATEST CODE UPDATED 29 Apr 2021
# Code for calculating applicability domain metric based on Tanimoto similarity

# Note google colab disconnects and clears data after 12 hrs of inactivity
# But code still runs in the background even if runtime is disconnected before the 12hr mark

# Relevant imports
import numpy as np
import pandas as pd # uses pandas python module to view and analyse data

import time
from time import strftime, gmtime

import os
from os import mkdir

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import MACCSkeys
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem import rdqueries

from rdkit import DataStructs

import requests
import random


#=======================================================================================#

#====================================================================================#
# Get list of random numbers
# Function to generate list of random integers with a specified number of elements in list

# random.randint(0,1) generates a random integer between 0 and 1 inclusive
def random_list(start,end,quota):
    rand_ls = []
    rand_ls.clear()

    while len(rand_ls) < quota:    
        rand_ls.append(random.randint(start,end))
        rand_ls = list(set(rand_ls))

    return rand_ls


# Function to get list of smiles and inchis given search property
def PubChem_get(PubChem_CID_ls):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/property/CanonicalSMILES/JSON'
    content = {'cid' : PubChem_CID_ls}
    r = requests.post(url, data = content, headers = { 'Content-Type': 'application/x-www-form-urlencoded' }).json()
    #print(r)
    smiles_ls = []
    smiles_ls.clear()

    # new_inchi_ls = []
    # new_inchi_ls.clear()
    try:
        smiles_count = len(r['PropertyTable']['Properties'])
        for x in range(0,smiles_count):

            query = r['PropertyTable']['Properties'][x]['CanonicalSMILES']
            #query2 = r['PropertyTable']['Properties'][x]['InChIKey']
            smiles_ls.append(query)
            #new_inchi_ls.append(query2)

    except:
        pass
    
    return smiles_ls
    
def Timecheck():
    start_time = strftime("%H:%M:%S", gmtime())
    print('\nCurrent time:')
    print(start_time)
    return

def Timer(start_time,end_time):
    hr, mod = divmod(end_time - start_time, 3600)
    min, sec = divmod(mod, 60)
    output = "{:0>2}:{:0>2}:{:05.2f}".format(int(hr),int(min),int(sec))
    output = str(output)

    return output
#====================================================================================#
# Fingerprint code from usual place from Tim's paper
def get_fingerprint(smiles):
    '''smiles dataframe'''
    rdkit_molecules=[Chem.MolFromSmiles(x) for x in smiles['Smiles']]
    rdkit_fingerprint=[]
    count = 0
    for mol in rdkit_molecules:
        # if count % 1000 == 0:
        #     print('Now fingerprinting {} of {}'.format(count,len(rdkit_molecules)))
        bit_info={}
        fp=rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=fingerprint_radius, nBits=fingerprint_length, \
                                                                      bitInfo=bit_info)
        
        rdkit_fingerprint.append(fp)
        count += 1
        
    #fingerprint_df=pd.DataFrame([np.array(list(x)).astype(int) for x in rdkit_fingerprint])
    fingerprint_df = pd.DataFrame(rdkit_fingerprint,columns=['BV'])
    
    return fingerprint_df

def CreateBitString(df,nBits):
    # Combine all columns with fingerprint values into format suitable for bitvector processing
    fp = df.iloc[:,0:nBits]
    fp['combined'] = fp[fp.columns.tolist()].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
    fp = fp[['combined']]

    return fp

def CreateBitVect(df):
    #print('\nCreateBitVect')

    df['BV'] = df.apply(lambda row : DataStructs.CreateFromBitString(row['combined']), axis = 1)
    df = df[['BV']]
    return df

def carbon_atom_check(smiles):
    # Check no. of C atoms
    query = rdqueries.AtomNumEqualsQueryAtom(6)

    mol = Chem.MolFromSmiles(smiles) 
    check = len(mol.GetAtomsMatchingQuery(query))

    return check


def Smiles_Checker(df):
    error_ls = []
    error_ls.clear()

    for x in range(0,len(df)):
        query = df.iloc[x]['Smiles']
        # List of meetals to remove because rdkit doesn't do it well enough
        # Check for metals and remove if necessary
        check_ls = ['Hg','Pb','He','Li','Be','Ne','Na','Mg','Al','Ar','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Kr','Rb','Se',
                'Sr','Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Pt','Ag','Au','Cd','In','Sn','Xe','Cs','Ba','La','Sb','Te','Os','Ir','As','Ge']

        if any(ele in query for ele in check_ls) == True:
            error_ls.append(x)
            continue

        # Check for salts and remove if necessary
        try:
            remover = SaltRemover()
            mol = Chem.MolFromSmiles(query)
            res = remover.StripMol(mol,dontRemoveEverything=True)
            update = Chem.MolToSmiles(res)
            query = update
        except:
            error_ls.append(x)
            continue

        # Check for at least one carbon atom since we want organics
        check = carbon_atom_check(query)

        # Has carbon atom ie. check != 0
        if check == 0:
            error_ls.append(x)
            continue

    # After performing all 3 checks per smiles in df and dropping if necessary
    # If all three checks passed, df will contain 2 smiles, else less than 2
    df = df.drop(error_ls)
    df = df.reset_index(drop=True)
    return df

def Min_df_check(df1,df2):
    if len(df1) < len(df2):
        return len(df1)
    if len(df1) > len(df2):
        return len(df2)
    else:   # If both dfs are equal
        return len(df1)

def PubChem_quota_loop(df):

    while len(df) != quota:
        Timecheck()

        smiles_to_get = quota - len(df)
        print('\ndf NOW CONTAINS {} SMILES'.format(len(df)))

        if quota <= 100000:
            PubChem_CID_ls = list(np.random.randint(low=1, high=100000000, size=(smiles_to_get)))

        if quota > 100000:
            check_count = quota - len(df)
            if check_count <= 100000: 
                PubChem_CID_ls = list(np.random.randint(low=1, high=100000000, size=check_count))
            if check_count > 100000:
                PubChem_CID_ls = list(np.random.randint(low=1, high=100000000, size=100000))

        PubChem_CID_ls = str(PubChem_CID_ls)
        PubChem_CID_ls = PubChem_CID_ls[1:-1].replace(" ", "")

        smiles_ls_update = PubChem_get(PubChem_CID_ls)
        smiles_df_update = pd.DataFrame(smiles_ls_update, columns=['Smiles'])
        smiles_df_update = Smiles_Checker(smiles_df_update)

        df = pd.concat([df,smiles_df_update],axis=0)
        df = df.drop_duplicates(subset='Smiles',keep='first')
        df = df.reset_index(drop=True)

    return df
#===================================================================================================#
# Define some more functions

# Function splits a df into actives and inactives and returns two dfs
def active_inactive_split(df):
    #print('\nactive_inactive_split CALLED')

    active_index = []
    active_index.clear()

    # Get index of actives
    temp = df.iloc[:,-2:]   # This will get SMILES column and activity column
    temp_df = df
    for x in range(0,len(temp)):
        if temp.iloc[x]['Binary Activity'] == 1:
            active_index.append(x)
    
    # Seperate actives and inactives intwo two dfs
    #print('\nCREATING ACTIVE DF...')
    active_df = temp_df.iloc[active_index]    
    active_df = active_df.reset_index(drop=True)

    #print('\nCREATING INACTIVE DF...')
    inactive_df = df.drop(df.index[[active_index]])
    inactive_df = inactive_df.reset_index(drop=True)
    
    return active_df, inactive_df


def Calc_Similarity(bv1,bv2,process,allsim):    # allsim defines whether to calculate for all similarity pairs
    #print('\nCalc_Similarity CALLED')
    # Calculate Tanimoto similarity between both samples
    output_dict = {}
    
    output_ls = []
    output_ls.clear()

    output_ls2 = []
    output_ls2.clear()

    output_ls3 = []
    output_ls3.clear()  

    output_ls4 = []
    output_ls4.clear()    

    output_ls5 = []
    output_ls5.clear()      

    sim_count = 0    

    for x in range(0,len(bv1)):
        check = 0
        bv1_bv = bv1.iloc[x]['BV']

        if x % 500 == 0:
            print ('Current time:')
            print(strftime("%H:%M:%S", gmtime()))
            print('NOW CALCULATING SIMILARITIES FOR PROCESS {} INDEX {}'.format(process,x))

        for y in range(0,len(bv2)):

            bv2_bv = bv2.iloc[y]['BV']
            sim = DataStructs.TanimotoSimilarity(bv1_bv,bv2_bv)
            output_ls.append(sim)
            output_ls2.append(x)
            output_ls3.append(y)
#             if process == 1:
#                 output_ls4.append(AOP_smiles.iloc[x]['SMILES'])
#                 output_ls5.append(target_smiles.iloc[y]['SMILES'])
#             else:
#                 output_ls4.append(target_smiles.iloc[x]['SMILES'])
#                 output_ls5.append(AOP_smiles.iloc[y]['SMILES'])

            # Process sim per molecule and determine if molecule is similar to second dataset                            
            if check == 0:
                if sim >= T_sim_threshold and allsim == 1: 
                    sim_count += 1
                    check += 1
                if sim >= T_sim_threshold and allsim == 0: 
                    sim_count += 1
                    check += 1
                    break              
    
    #print('\nTANIMOTO SIMILARITIES CALCULATED')
    # Put all required values into dictionary
    output_dict['sim_count'] = sim_count
    output_dict['output_ls'] = output_ls
    output_dict['bv1_index'] = output_ls2
    output_dict['bv2_index'] = output_ls3
#     output_dict['AOP SMILES'] = output_ls4
#     output_dict['Target SMILES'] = output_ls5
    #print(output_dict.get('sim_count'))

    return output_dict

def Remove_Unnamed(df):
    try:
        df = df.drop(['Unnamed: 0'],axis=1)
    except:
        pass
    
    return df
#===================================================================================================#

#===================================================================================================#
# Start main code

sim_ls = []
sim_ls.clear()

mol_threshold_ls = []
mol_threshold_ls.clear()

temp_ls = []
temp_ls.clear()

# READ REQUIRED FILES
print('\nSETTING UP TARGETS...')

# Set up targets using Tim's data
# Read csv file containing targets to calculate
filename = 'C:/Users/mwhw3/Desktop/AOP project/Input data/'
filename = filename + 'Targets to calculate 5.csv'

target_df = pd.read_csv(filename)
target_df = target_df[['Target']]
print(target_df)

#====================================================================================#
nBits = 2048
length = nBits
fingerprint_length = nBits
quota = 10000
sample_ls = [1,2,3]
file_active = 0
fingerprint_radius = 2

for sample in sample_ls:
    # Set up initial df
    print('\nSETTING UP PubChem DF1...')
    start_time = time.time()
    Timecheck()
    if quota <= 100000:
        PubChem_CID_ls = list(np.random.randint(low=1, high=100000000, size=(quota)))

    if quota > 100000:
        PubChem_CID_ls = list(np.random.randint(low=1, high=100000000, size=(100000)))

    PubChem_CID_ls = str(PubChem_CID_ls)
    PubChem_CID_ls = PubChem_CID_ls[1:-1].replace(" ", "")

    smiles_ls = PubChem_get(PubChem_CID_ls)
    smiles_df1 = pd.DataFrame(smiles_ls, columns=['Smiles'])
    smiles_df1 = Smiles_Checker(smiles_df1)

    # Repeat random generation until quota is reached
    smiles_df1 = PubChem_quota_loop(smiles_df1)

    print(smiles_df1)
    Timecheck()

    print('\nPubChem DF1 SET UP')

    # Save file containing smiles 
    root_dir = 'C:/Users/mwhw3/Desktop/AOP project/PubChem smiles/'
    filename = root_dir + str(quota) + ' PubChem smiles_' + str(nBits) + ' bits_sample ' + str(sample) + '.csv'
    smiles_df1.to_csv(filename)

    #====================================================================================#
    # Converting smiles to fingerprints
    training_df = get_fingerprint(smiles_df1)

    print(training_df)

    #===================================================================================================#
    print ('Current time:')
    print(strftime("%H:%M:%S", gmtime())) 

    target_count = 0
    train_inactive_bv_df = training_df

    print (train_inactive_bv_df.shape)

    initial = 0

    for protein in range(initial,len(target_df)):

        T_ls = [0.3]
        nBits = 2048

        target = str(target_df.loc[protein]['Target'])

        print('Processing target: {}'.format(protein))
        print('Processing target: {}'.format(target))

        # Read file 2 (test dataset)
        print ('\nReading file 2...')
        print ('Current time:')
        print(strftime("%H:%M:%S", gmtime())) 
        start = time.time()

        filename = 'C:/Users/mwhw3/Desktop/AOP project/Input data/'
        filename = filename + str(target) + '_train_fingerprints Morgan ' + str(length) + '.csv'
        test_fp_df = pd.read_csv(filename)
        test_fp_df = Remove_Unnamed(test_fp_df)

        test_fp_df = test_fp_df.reset_index(drop=True)

        print (test_fp_df.shape)

        # Split training data into actives and inactives
        test_active, test_inactive = active_inactive_split(test_fp_df)
        if file_active == 0:    # Lazy man way of choosing inactives without replacing code
            test_active = test_inactive

        target_smiles = test_active[['SMILES']]
        print(target_smiles)
        test_active = test_active.drop(['SMILES'],axis=1)

        # Drop Binary activity column
        test_active = test_active.iloc[:, :-1]
        print(test_active)

        end = time.time()
        elapsed = end - start
        minutes = elapsed // 60
        seconds = elapsed - (minutes*60)
        print('File 2 took {} minutes and {} seconds to read'.format(minutes,seconds))

        #=========================================================================================#

        print('\nCREATING BITVECT DFs FOR TEST DATA...')
        test_active_fp = CreateBitString(test_active,2048)
        test_active_bv_df = CreateBitVect(test_active_fp)
        # test_inactive_bv_df = CreateBitVect(test_inactive_fp)

        print('\nTEST BITVECT DFs CREATED')
        #=========================================================================================#

        # Caclulate Tanimoto similarities
        # Slow process
        print('\nCALCULATING TANIMOTO SIMILARITIES...')

        # Set similarity threshold

        sim_ls = []
        sim_ls.clear()

        temp_ls = []
        temp_ls.clear()

        T_sim_threshold = float(T_ls[0])

        print('\nNOW CALCULATING FOR THRESHOLD: {}'.format(T_sim_threshold))
        print('\nNOW CALCULATING FOR TARGET: {}'.format(target))

        indicator_ls = []
        indicator_ls.clear()

        train_count_ls = []
        train_count_ls.clear()

        test_count_ls = []
        test_count_ls.clear()

        # Ensure that training bv df is followed by teste bv df for train sim indicator
        # Ensure that actives are matched with actives and vice versa
        # Start multiprocessing and queues
        start = time.time()

        temp_dict1 = Calc_Similarity(train_inactive_bv_df,test_active_bv_df,1,1)
        temp_dict2 = Calc_Similarity(test_active_bv_df,train_inactive_bv_df,2,0) 
        print('\nSIMILARITY CALCULATIONS FINISHED')
        # Get sim_count for training with test
        temp_count1 = temp_dict1.get('sim_count')
        temp_count2 = temp_dict2.get('sim_count')

        temp_ls1 = temp_dict1.get('output_ls')
        temp_ls2 = temp_dict2.get('output_ls')

        temp_ls3 = temp_dict1.get('bv1_index')
        temp_ls4 = temp_dict1.get('bv2_index')

        temp_ls5 = temp_dict2.get('bv1_index')
        temp_ls6 = temp_dict2.get('bv2_index')

        temp_ls7 = temp_dict1.get('AOP SMILES')
        temp_ls8 = temp_dict1.get('Target SMILES') 

        temp_ls9 = temp_dict2.get('AOP SMILES')
        temp_ls10 = temp_dict2.get('Target SMILES')            

        sim_count = temp_count1 + temp_count2

        end = time.time()
        elapsed = end - start
        minutes = elapsed // 60
        seconds = elapsed - (minutes*60)

        print('\n#=====================================================================#')
        print('\n#=====================================================================#')
        print('\nTRAINING AND TEST TANIMOTO SIMILARITIES CALCULATED')
        print('Code took {} minutes and {} seconds'.format(minutes,seconds))
        print('\n#=====================================================================#')
        print('\n#=====================================================================#')

        #=========================================================================================#
        print('\nCALCULATING TRAINING SIMILARITY INDICATOR...')

        train_indicator = temp_count1 / (len(train_inactive_bv_df)) * 100
        indicator_ls.append(train_indicator)

        print('\nTRAINING SIMILARITY INDICATOR CALCULATED')
        #=========================================================================================#

        print('\nCALCULATING TEST SIMILARITY INDICATOR...')

        test_indicator =  temp_count2 / (len(test_active_bv_df)) * 100
        indicator_ls.append(test_indicator)

        print('\nTEST SIMILARITY INDICATOR CALCULATED')

        #=========================================================================================#
        print('\nCALCULATING AVERAGE SIMILARITY INDICATOR...')

        average_indicator = (train_indicator + test_indicator) / 2
        indicator_ls.append(average_indicator)

        indicator_df = pd.DataFrame(indicator_ls).T
        indicator_df.columns = ['Train','Test','Average']

        print(indicator_df)

        print('\nAVERAGE SIMILARITY INDICATOR CALCULATED')

        #=========================================================================================#

        # Append data counts in dfs
        train_count_ls.append((len(train_inactive_bv_df)))
        test_count_ls.append((len(test_active_bv_df)))
        sim_ls.append(T_sim_threshold)
        temp_ls.append(target)

        #=========================================================================================#
        # Collate all files/results
        # Quick process

        combined_df = indicator_df

        combined_df['Sim'] = sim_ls
        combined_df['Target'] = temp_ls
        combined_df['train_target count'] = train_count_ls
        combined_df['test_target count'] = test_count_ls
        print(combined_df)

        #===========================================================================#
        # Process df and save

        print('\nSAVING DF FOR TARGET {}'.format(protein))
        print ('\nSAVING FILES...')

        if protein == 0:
            all_df = combined_df
        else:
            all_df = pd.concat([all_df,combined_df],axis=0)
            all_df = all_df.reset_index(drop=True)

        end = time.time()
        elapsed = end - start
        minutes = elapsed // 60
        seconds = elapsed - (minutes*60)

        print ('\n#=========================================================================#')

        # Save all_df in separate easy to access folder (combined df with all calculated targets for easy copying)
        root_dir = 'C:/Users/mwhw3/Desktop/AOP project/Similarity results/'

        if file_active == 0:
            filename = root_dir + 'all_df_quota ' + str(quota) + '_sample '+ str(sample) + '_inactivedata_indicator'   
        if file_active == 1:  
            filename = root_dir + 'all_df_quota ' + str(quota) + '_sample '+ str(sample) + '_activedata_indicator'    

        filename = filename + str(nBits) + 'bits_' + str(T_sim_threshold) + 'T_threshold.csv'
        all_df.to_csv(filename)
        print('\nALL RESULTS FILE SAVED')


print('\nFINISHED')


SETTING UP TARGETS...
     Target
0      AChE
1   ADORA2A
2    ADRA2A
3     ADRB1
4     ADRB2
..      ...
74   SLC6A3
75      SRC
76    TACR2
77   TBXA2R
78      TEK

[79 rows x 1 columns]

SETTING UP PubChem DF1...

Current time:
11:07:34

Current time:
11:07:45

df NOW CONTAINS 4952 SMILES

Current time:
11:07:54

df NOW CONTAINS 9938 SMILES

Current time:
11:07:55

df NOW CONTAINS 9998 SMILES
                                                 Smiles
0     CCN(CC)C(=O)C(C)OC(=O)CSC1=NN=C(N1C2=CC=C(C=C2...
1             CC1=CC(=C(C=C1)OC)C(C)NC2CC(=O)N(C2)C3CC3
2     CC(C)C1=CC=CC=C1N2C(=NN(C2=S)CN(C)CC3=CC=C(C=C...
3        CC1=C(C(=NN1C2=CC=CC=C2)C)C3=NC4=C(N3C)C=CN=C4
4                   CC(C)CN(CC(C1=CC(=CC=C1)N)O)C2CCCC2
...                                                 ...
9995  C1CN(CCN(CCN(CCN1CC(=O)NC(CC2=CC=CC=C2)C(=O)CC...
9996  CC(C)(C)OC(=O)N1CCCC1C2=NC3(COC3)C4=C(N2)C=C(C...
9997  C1=CC(=C(C=C1C(F)(F)F)NC(=O)COC(=O)COC2=C(C=C(...
9998  CN(C1=CC(=CC(=C1)C(=O)NC2CCNC2CC3=

C:\Users\mwhw3\Anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\indexes\base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


                                SMILES
0                Brc1ccc(Oc2ccccc2)cc1
1      C#CC(C)(C)OC[n+]1ccn(C)c1/C=N/O
2     C#CC(C)C(C)OC[n+]1ccn(C)c1/C=N/O
3         C#CC(C)OC[n+]1ccn(C)c1/C=N/O
4                     C#CC(O)C(CC)CCCC
...                                ...
1462      c1ccc2cc3c(ccc4ccccc43)cc2c1
1463              c1ccc2cc3ccccc3cc2c1
1464                    c1ccc2ccccc2c1
1465                     c1ccc2occc2c1
1466                        c1nc[nH]n1

[1467 rows x 1 columns]
      0  1  2  3  4  5  6  7  8  9  ...  2038  2039  2040  2041  2042  2043  \
0     0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
1     0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
2     0  1  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
3     0  1  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
4     0  1  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     1   
...  .. .. .. .. .. .. .. .. .. .

C:\Users\mwhw3\Anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)



TEST BITVECT DFs CREATED

CALCULATING TANIMOTO SIMILARITIES...

NOW CALCULATING FOR THRESHOLD: 0.3

NOW CALCULATING FOR TARGET: AChE
Current time:
11:08:01
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 0
Current time:
11:08:40
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 500
Current time:
11:09:19
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1000
Current time:
11:09:57
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1500
Current time:
11:10:36
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2000
Current time:
11:11:15
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2500
Current time:
11:11:54
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3000
Current time:
11:12:32
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3500
Current time:
11:13:11
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4000
Current time:
11:13:50
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4500
Current time:
11:14:28
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 5000
Current time:
11:15:07


...

                                                 SMILES
0                                            B.C1COCCN1
1                                                 BrCBr
2                                      BrCC(Br)c1ccccc1
3                                              BrCCCCBr
4     Brc1c(Br)c(Br)c(Oc2c(Br)c(Br)c(Br)c(Br)c2Br)c(...
...                                                 ...
1405                                      c1ccc2occc2c1
1406                c1ccc2sc(SN(C3CCCCC3)C3CCCCC3)nc2c1
1407                          c1ccc2sc(SNC3CCCCC3)nc2c1
1408                          c1ccc2sc(SSN3CCOCC3)nc2c1
1409                                         c1nc[nH]n1

[1410 rows x 1 columns]
      0  1  2  3  4  5  6  7  8  9  ...  2038  2039  2040  2041  2042  2043  \
0     0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
1     0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
2     0  1  0  0  0  0  0  0  0  0  ...     0     0     0     0   

C:\Users\mwhw3\Anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)



TEST BITVECT DFs CREATED

CALCULATING TANIMOTO SIMILARITIES...

NOW CALCULATING FOR THRESHOLD: 0.3

NOW CALCULATING FOR TARGET: TBXA2R
Current time:
05:46:21
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 0
Current time:
05:46:58
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 500
Current time:
05:47:35
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1000
Current time:
05:48:13
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1500
Current time:
05:48:50
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2000
Current time:
05:49:27
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2500
Current time:
05:50:04
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3000
Current time:
05:50:42
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3500
Current time:
05:51:19
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4000
Current time:
05:51:56
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4500
Current time:
05:52:33
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 5000
Current time:
05:53:1

C:\Users\mwhw3\Anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\indexes\base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
C:\Users\mwhw3\Anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)



TEST BITVECT DFs CREATED

CALCULATING TANIMOTO SIMILARITIES...

NOW CALCULATING FOR THRESHOLD: 0.3

NOW CALCULATING FOR TARGET: TEK
Current time:
06:04:18
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 0
Current time:
06:04:41
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 500
Current time:
06:05:04
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1000
Current time:
06:05:27
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1500
Current time:
06:05:49
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2000
Current time:
06:06:12
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2500
Current time:
06:06:35
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3000
Current time:
06:06:58
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3500
Current time:
06:07:20
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4000
Current time:
06:07:43
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4500
Current time:
06:08:06
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 5000
Current time:
06:08:29
N

In [9]:
# For use with model vs other targets from Tim's code
# LATEST CODE UPDATED 29 Apr 2021
# Code for calculating applicability domain metric based on Tanimoto similarity

# Note google colab disconnects and clears data after 12 hrs of inactivity
# But code still runs in the background even if runtime is disconnected before the 12hr mark

# Relevant imports
import numpy as np
import pandas as pd # uses pandas python module to view and analyse data

import time
from time import strftime, gmtime

import os
from os import mkdir

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import MACCSkeys
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem import rdqueries

from rdkit import DataStructs

import requests
import random


#=======================================================================================#

#====================================================================================#
# Get list of random numbers
# Function to generate list of random integers with a specified number of elements in list

# random.randint(0,1) generates a random integer between 0 and 1 inclusive
def random_list(start,end,quota):
    rand_ls = []
    rand_ls.clear()

    while len(rand_ls) < quota:    
        rand_ls.append(random.randint(start,end))
        rand_ls = list(set(rand_ls))

    return rand_ls


# Function to get list of smiles and inchis given search property
def PubChem_get(PubChem_CID_ls):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/property/CanonicalSMILES/JSON'
    content = {'cid' : PubChem_CID_ls}
    r = requests.post(url, data = content, headers = { 'Content-Type': 'application/x-www-form-urlencoded' }).json()
    #print(r)
    smiles_ls = []
    smiles_ls.clear()

    # new_inchi_ls = []
    # new_inchi_ls.clear()
    try:
        smiles_count = len(r['PropertyTable']['Properties'])
        for x in range(0,smiles_count):

            query = r['PropertyTable']['Properties'][x]['CanonicalSMILES']
            #query2 = r['PropertyTable']['Properties'][x]['InChIKey']
            smiles_ls.append(query)
            #new_inchi_ls.append(query2)

    except:
        pass
    
    return smiles_ls
    
def Timecheck():
    start_time = strftime("%H:%M:%S", gmtime())
    print('\nCurrent time:')
    print(start_time)
    return

def Timer(start_time,end_time):
    hr, mod = divmod(end_time - start_time, 3600)
    min, sec = divmod(mod, 60)
    output = "{:0>2}:{:0>2}:{:05.2f}".format(int(hr),int(min),int(sec))
    output = str(output)

    return output
#====================================================================================#
# Fingerprint code from usual place from Tim's paper
def get_fingerprint(smiles):
    '''smiles dataframe'''
    rdkit_molecules=[Chem.MolFromSmiles(x) for x in smiles['Smiles']]
    rdkit_fingerprint=[]
    count = 0
    for mol in rdkit_molecules:
        # if count % 1000 == 0:
        #     print('Now fingerprinting {} of {}'.format(count,len(rdkit_molecules)))
        bit_info={}
        fp=rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=fingerprint_radius, nBits=fingerprint_length, \
                                                                      bitInfo=bit_info)
        
        rdkit_fingerprint.append(fp)
        count += 1
        
    #fingerprint_df=pd.DataFrame([np.array(list(x)).astype(int) for x in rdkit_fingerprint])
    fingerprint_df = pd.DataFrame(rdkit_fingerprint,columns=['BV'])
    
    return fingerprint_df

def CreateBitString(df,nBits):
    # Combine all columns with fingerprint values into format suitable for bitvector processing
    fp = df.iloc[:,0:nBits]
    fp['combined'] = fp[fp.columns.tolist()].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
    fp = fp[['combined']]

    return fp

def CreateBitVect(df):
    #print('\nCreateBitVect')

    df['BV'] = df.apply(lambda row : DataStructs.CreateFromBitString(row['combined']), axis = 1)
    df = df[['BV']]
    return df

def carbon_atom_check(smiles):
    # Check no. of C atoms
    query = rdqueries.AtomNumEqualsQueryAtom(6)

    mol = Chem.MolFromSmiles(smiles) 
    check = len(mol.GetAtomsMatchingQuery(query))

    return check


def Smiles_Checker(df):
    error_ls = []
    error_ls.clear()

    for x in range(0,len(df)):
        query = df.iloc[x]['Smiles']
        # List of meetals to remove because rdkit doesn't do it well enough
        # Check for metals and remove if necessary
        check_ls = ['Hg','Pb','He','Li','Be','Ne','Na','Mg','Al','Ar','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Kr','Rb','Se',
                'Sr','Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Pt','Ag','Au','Cd','In','Sn','Xe','Cs','Ba','La','Sb','Te','Os','Ir','As','Ge']

        if any(ele in query for ele in check_ls) == True:
            error_ls.append(x)
            continue

        # Check for salts and remove if necessary
        try:
            remover = SaltRemover()
            mol = Chem.MolFromSmiles(query)
            res = remover.StripMol(mol,dontRemoveEverything=True)
            update = Chem.MolToSmiles(res)
            query = update
        except:
            error_ls.append(x)
            continue

        # Check for at least one carbon atom since we want organics
        check = carbon_atom_check(query)

        # Has carbon atom ie. check != 0
        if check == 0:
            error_ls.append(x)
            continue

    # After performing all 3 checks per smiles in df and dropping if necessary
    # If all three checks passed, df will contain 2 smiles, else less than 2
    df = df.drop(error_ls)
    df = df.reset_index(drop=True)
    return df

def Min_df_check(df1,df2):
    if len(df1) < len(df2):
        return len(df1)
    if len(df1) > len(df2):
        return len(df2)
    else:   # If both dfs are equal
        return len(df1)

def PubChem_quota_loop(df):

    while len(df) != quota:
        Timecheck()

        smiles_to_get = quota - len(df)
        print('\ndf NOW CONTAINS {} SMILES'.format(len(df)))

        if quota <= 100000:
            PubChem_CID_ls = list(np.random.randint(low=1, high=100000000, size=(smiles_to_get)))

        if quota > 100000:
            check_count = quota - len(df)
            if check_count <= 100000: 
                PubChem_CID_ls = list(np.random.randint(low=1, high=100000000, size=check_count))
            if check_count > 100000:
                PubChem_CID_ls = list(np.random.randint(low=1, high=100000000, size=100000))

        PubChem_CID_ls = str(PubChem_CID_ls)
        PubChem_CID_ls = PubChem_CID_ls[1:-1].replace(" ", "")

        smiles_ls_update = PubChem_get(PubChem_CID_ls)
        smiles_df_update = pd.DataFrame(smiles_ls_update, columns=['Smiles'])
        smiles_df_update = Smiles_Checker(smiles_df_update)

        df = pd.concat([df,smiles_df_update],axis=0)
        df = df.drop_duplicates(subset='Smiles',keep='first')
        df = df.reset_index(drop=True)

    return df
#===================================================================================================#
# Define some more functions

# Function splits a df into actives and inactives and returns two dfs
def active_inactive_split(df):
    #print('\nactive_inactive_split CALLED')

    active_index = []
    active_index.clear()

    # Get index of actives
    temp = df.iloc[:,-2:]   # This will get SMILES column and activity column
    temp_df = df
    for x in range(0,len(temp)):
        if temp.iloc[x]['Binary Activity'] == 1:
            active_index.append(x)
    
    # Seperate actives and inactives intwo two dfs
    #print('\nCREATING ACTIVE DF...')
    active_df = temp_df.iloc[active_index]    
    active_df = active_df.reset_index(drop=True)

    #print('\nCREATING INACTIVE DF...')
    inactive_df = df.drop(df.index[[active_index]])
    inactive_df = inactive_df.reset_index(drop=True)
    
    return active_df, inactive_df


def Calc_Similarity(bv1,bv2,process,allsim):    # allsim defines whether to calculate for all similarity pairs
    #print('\nCalc_Similarity CALLED')
    # Calculate Tanimoto similarity between both samples
    output_dict = {}
    
    output_ls = []
    output_ls.clear()

    output_ls2 = []
    output_ls2.clear()

    output_ls3 = []
    output_ls3.clear()  

    output_ls4 = []
    output_ls4.clear()    

    output_ls5 = []
    output_ls5.clear()      

    sim_count = 0    

    for x in range(0,len(bv1)):
        check = 0
        bv1_bv = bv1.iloc[x]['BV']

        if x % 500 == 0:
            print ('Current time:')
            print(strftime("%H:%M:%S", gmtime()))
            print('NOW CALCULATING SIMILARITIES FOR PROCESS {} INDEX {}'.format(process,x))

        for y in range(0,len(bv2)):

            bv2_bv = bv2.iloc[y]['BV']
            sim = DataStructs.TanimotoSimilarity(bv1_bv,bv2_bv)
            output_ls.append(sim)
            output_ls2.append(x)
            output_ls3.append(y)
#             if process == 1:
#                 output_ls4.append(AOP_smiles.iloc[x]['SMILES'])
#                 output_ls5.append(target_smiles.iloc[y]['SMILES'])
#             else:
#                 output_ls4.append(target_smiles.iloc[x]['SMILES'])
#                 output_ls5.append(AOP_smiles.iloc[y]['SMILES'])

            # Process sim per molecule and determine if molecule is similar to second dataset                            
            if check == 0:
                if sim >= T_sim_threshold and allsim == 1: 
                    sim_count += 1
                    check += 1
                if sim >= T_sim_threshold and allsim == 0: 
                    sim_count += 1
                    check += 1
                    break              
    
    #print('\nTANIMOTO SIMILARITIES CALCULATED')
    # Put all required values into dictionary
    output_dict['sim_count'] = sim_count
    output_dict['output_ls'] = output_ls
    output_dict['bv1_index'] = output_ls2
    output_dict['bv2_index'] = output_ls3
#     output_dict['AOP SMILES'] = output_ls4
#     output_dict['Target SMILES'] = output_ls5
    #print(output_dict.get('sim_count'))

    return output_dict

def Remove_Unnamed(df):
    try:
        df = df.drop(['Unnamed: 0'],axis=1)
    except:
        pass
    
    return df
#===================================================================================================#

#===================================================================================================#
# Start main code

sim_ls = []
sim_ls.clear()

mol_threshold_ls = []
mol_threshold_ls.clear()

temp_ls = []
temp_ls.clear()

# READ REQUIRED FILES
print('\nSETTING UP TARGETS...')

# Set up targets using Tim's data
# Read csv file containing targets to calculate
filename = 'C:/Users/mwhw3/Desktop/AOP project/Input data/'
filename = filename + 'Targets to calculate 5.csv'

target_df = pd.read_csv(filename)
target_df = target_df[['Target']]
print(target_df)

#====================================================================================#
nBits = 2048
length = nBits
fingerprint_length = nBits
quota = 10000
sample_ls = [1,2,3]
file_active = 1
fingerprint_radius = 2

for sample in sample_ls:
    # Set up initial df
    print('\nSETTING UP PubChem DF1...')
    start_time = time.time()
    Timecheck()


    # Repeat random generation until quota is reached
    root_dir = 'C:/Users/mwhw3/Desktop/AOP project/PubChem smiles/'
    filename = root_dir + str(quota) + ' PubChem smiles_' + str(nBits) + ' bits_sample ' + str(sample) + '.csv'
    smiles_df1 = pd.read_csv(filename)
    smiles_df1 = Remove_Unnamed(smiles_df1)

    print(smiles_df1)
    Timecheck()

    print('\nPubChem DF1 SET UP')
    #====================================================================================#
    # Converting smiles to fingerprints
    training_df = get_fingerprint(smiles_df1)

    print(training_df)

    #===================================================================================================#
    print ('Current time:')
    print(strftime("%H:%M:%S", gmtime())) 

    target_count = 0
    train_inactive_bv_df = training_df

    print (train_inactive_bv_df.shape)

    initial = 0

    for protein in range(initial,len(target_df)):

        T_ls = [0.3]
        nBits = 2048

        target = str(target_df.loc[protein]['Target'])

        print('Processing target: {}'.format(protein))
        print('Processing target: {}'.format(target))

        # Read file 2 (test dataset)
        print ('\nReading file 2...')
        print ('Current time:')
        print(strftime("%H:%M:%S", gmtime())) 
        start = time.time()

        filename = 'C:/Users/mwhw3/Desktop/AOP project/Input data/'
        filename = filename + str(target) + '_train_fingerprints Morgan ' + str(length) + '.csv'
        test_fp_df = pd.read_csv(filename)
        test_fp_df = Remove_Unnamed(test_fp_df)

        test_fp_df = test_fp_df.reset_index(drop=True)

        print (test_fp_df.shape)

        # Split training data into actives and inactives
        test_active, test_inactive = active_inactive_split(test_fp_df)
        if file_active == 0:    # Lazy man way of choosing inactives without replacing code
            test_active = test_inactive

        target_smiles = test_active[['SMILES']]
        print(target_smiles)
        test_active = test_active.drop(['SMILES'],axis=1)

        # Drop Binary activity column
        test_active = test_active.iloc[:, :-1]
        print(test_active)

        end = time.time()
        elapsed = end - start
        minutes = elapsed // 60
        seconds = elapsed - (minutes*60)
        print('File 2 took {} minutes and {} seconds to read'.format(minutes,seconds))

        #=========================================================================================#

        print('\nCREATING BITVECT DFs FOR TEST DATA...')
        test_active_fp = CreateBitString(test_active,2048)
        test_active_bv_df = CreateBitVect(test_active_fp)
        # test_inactive_bv_df = CreateBitVect(test_inactive_fp)

        print('\nTEST BITVECT DFs CREATED')
        #=========================================================================================#

        # Caclulate Tanimoto similarities
        # Slow process
        print('\nCALCULATING TANIMOTO SIMILARITIES...')

        # Set similarity threshold

        sim_ls = []
        sim_ls.clear()

        temp_ls = []
        temp_ls.clear()

        T_sim_threshold = float(T_ls[0])

        print('\nNOW CALCULATING FOR THRESHOLD: {}'.format(T_sim_threshold))
        print('\nNOW CALCULATING FOR TARGET: {}'.format(target))

        indicator_ls = []
        indicator_ls.clear()

        train_count_ls = []
        train_count_ls.clear()

        test_count_ls = []
        test_count_ls.clear()

        # Ensure that training bv df is followed by teste bv df for train sim indicator
        # Ensure that actives are matched with actives and vice versa
        # Start multiprocessing and queues
        start = time.time()

        temp_dict1 = Calc_Similarity(train_inactive_bv_df,test_active_bv_df,1,1)
        temp_dict2 = Calc_Similarity(test_active_bv_df,train_inactive_bv_df,2,0) 
        print('\nSIMILARITY CALCULATIONS FINISHED')
        # Get sim_count for training with test
        temp_count1 = temp_dict1.get('sim_count')
        temp_count2 = temp_dict2.get('sim_count')

        temp_ls1 = temp_dict1.get('output_ls')
        temp_ls2 = temp_dict2.get('output_ls')

        temp_ls3 = temp_dict1.get('bv1_index')
        temp_ls4 = temp_dict1.get('bv2_index')

        temp_ls5 = temp_dict2.get('bv1_index')
        temp_ls6 = temp_dict2.get('bv2_index')

        temp_ls7 = temp_dict1.get('AOP SMILES')
        temp_ls8 = temp_dict1.get('Target SMILES') 

        temp_ls9 = temp_dict2.get('AOP SMILES')
        temp_ls10 = temp_dict2.get('Target SMILES')            

        sim_count = temp_count1 + temp_count2

        end = time.time()
        elapsed = end - start
        minutes = elapsed // 60
        seconds = elapsed - (minutes*60)

        print('\n#=====================================================================#')
        print('\n#=====================================================================#')
        print('\nTRAINING AND TEST TANIMOTO SIMILARITIES CALCULATED')
        print('Code took {} minutes and {} seconds'.format(minutes,seconds))
        print('\n#=====================================================================#')
        print('\n#=====================================================================#')

        #=========================================================================================#
        print('\nCALCULATING TRAINING SIMILARITY INDICATOR...')

        train_indicator = temp_count1 / (len(train_inactive_bv_df)) * 100
        indicator_ls.append(train_indicator)

        print('\nTRAINING SIMILARITY INDICATOR CALCULATED')
        #=========================================================================================#

        print('\nCALCULATING TEST SIMILARITY INDICATOR...')

        test_indicator =  temp_count2 / (len(test_active_bv_df)) * 100
        indicator_ls.append(test_indicator)

        print('\nTEST SIMILARITY INDICATOR CALCULATED')

        #=========================================================================================#
        print('\nCALCULATING AVERAGE SIMILARITY INDICATOR...')

        average_indicator = (train_indicator + test_indicator) / 2
        indicator_ls.append(average_indicator)

        indicator_df = pd.DataFrame(indicator_ls).T
        indicator_df.columns = ['Train','Test','Average']

        print(indicator_df)

        print('\nAVERAGE SIMILARITY INDICATOR CALCULATED')

        #=========================================================================================#

        # Append data counts in dfs
        train_count_ls.append((len(train_inactive_bv_df)))
        test_count_ls.append((len(test_active_bv_df)))
        sim_ls.append(T_sim_threshold)
        temp_ls.append(target)

        #=========================================================================================#
        # Collate all files/results
        # Quick process

        combined_df = indicator_df

        combined_df['Sim'] = sim_ls
        combined_df['Target'] = temp_ls
        combined_df['train_target count'] = train_count_ls
        combined_df['test_target count'] = test_count_ls
        print(combined_df)

        #===========================================================================#
        # Process df and save

        print('\nSAVING DF FOR TARGET {}'.format(protein))
        print ('\nSAVING FILES...')

        if protein == 0:
            all_df = combined_df
        else:
            all_df = pd.concat([all_df,combined_df],axis=0)
            all_df = all_df.reset_index(drop=True)

        end = time.time()
        elapsed = end - start
        minutes = elapsed // 60
        seconds = elapsed - (minutes*60)

        print ('\n#=========================================================================#')

        # Save all_df in separate easy to access folder (combined df with all calculated targets for easy copying)
        root_dir = 'C:/Users/mwhw3/Desktop/AOP project/Similarity results/'

        if file_active == 0:
            filename = root_dir + 'all_df_quota ' + str(quota) + '_sample '+ str(sample) + '_inactivedata_indicator'   
        if file_active == 1:  
            filename = root_dir + 'all_df_quota ' + str(quota) + '_sample '+ str(sample) + '_activedata_indicator'    

        filename = filename + str(nBits) + 'bits_' + str(T_sim_threshold) + 'T_threshold.csv'
        all_df.to_csv(filename)
        print('\nALL RESULTS FILE SAVED')


print('\nFINISHED')


SETTING UP TARGETS...
     Target
0      AChE
1   ADORA2A
2    ADRA2A
3     ADRB1
4     ADRB2
..      ...
74   SLC6A3
75      SRC
76    TACR2
77   TBXA2R
78      TEK

[79 rows x 1 columns]

SETTING UP PubChem DF1...

Current time:
06:15:16
                                                 Smiles
0     CCN(CC)C(=O)C(C)OC(=O)CSC1=NN=C(N1C2=CC=C(C=C2...
1             CC1=CC(=C(C=C1)OC)C(C)NC2CC(=O)N(C2)C3CC3
2     CC(C)C1=CC=CC=C1N2C(=NN(C2=S)CN(C)CC3=CC=C(C=C...
3        CC1=C(C(=NN1C2=CC=CC=C2)C)C3=NC4=C(N3C)C=CN=C4
4                   CC(C)CN(CC(C1=CC(=CC=C1)N)O)C2CCCC2
...                                                 ...
9995  C1CN(CCN(CCN(CCN1CC(=O)NC(CC2=CC=CC=C2)C(=O)CC...
9996  CC(C)(C)OC(=O)N1CCCC1C2=NC3(COC3)C4=C(N2)C=C(C...
9997  C1=CC(=C(C=C1C(F)(F)F)NC(=O)COC(=O)COC2=C(C=C(...
9998  CN(C1=CC(=CC(=C1)C(=O)NC2CCNC2CC3=CC=CC=C3)COC...
9999         CCN1C=C(C=N1)C(=O)NC2=NN(C=C2)CC3=CC=CC=C3

[10000 rows x 1 columns]

Current time:
06:15:16

PubChem DF1 SET UP
                 

C:\Users\mwhw3\Anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\indexes\base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


                                                 SMILES
0                   Brc1ccc(-c2nnc3sc(-c4ccoc4)nn23)cc1
1          Brc1ccc(CNc2ccnc(NC3CCN(Cc4ccccc4)CC3)n2)cc1
2                Brc1ccc2c(CCC3CCN(Cc4ccccc4)CC3)noc2c1
3                   Brc1cccc(-c2nnc3sc(-c4ccoc4)nn23)c1
4                    C#CC(C)NC1CCc2ccc(SC(=O)N(C)C)cc21
...                                                 ...
1999  c1cnc2nc3c(c(NCCCCCCNc4c5c(nc6ncccc46)CCCC5)c2...
2000  c1cncc(-c2cc3c(cn2)[nH]c2ncc(-c4ccc(CN5CCCCC5)...
2001  c1ncc(-c2cc3c(cn2)[nH]c2ncc(-c4ccc(CN5CCCCC5)c...
2002  c1ncc(-c2cc3c(cn2)[nH]c2ncc(-c4ccc(CN5CCCCC5)c...
2003  c1nnc(-c2cc3c(cn2)[nH]c2ncc(-c4ccc(CN5CCCCC5)c...

[2004 rows x 1 columns]
      0  1  2  3  4  5  6  7  8  9  ...  2038  2039  2040  2041  2042  2043  \
0     0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
1     0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
2     0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   

C:\Users\mwhw3\Anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)



TEST BITVECT DFs CREATED

CALCULATING TANIMOTO SIMILARITIES...

NOW CALCULATING FOR THRESHOLD: 0.3

NOW CALCULATING FOR TARGET: AChE
Current time:
06:15:23
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 0
Current time:
06:16:16
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 500
Current time:
06:17:09
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1000
Current time:
06:18:02
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1500
Current time:
06:18:55
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2000
Current time:
06:19:47
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2500
Current time:
06:20:40
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3000
Current time:
06:21:33
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3500
Current time:
06:22:26
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4000
Current time:
06:23:19
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4500
Current time:
06:24:12
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 5000
Current time:
06:25:05


...

                                                SMILES
0    C#CCCCCCNC(=O)c1coc(C2C3CCC(O3)C2Cc2ccccc2CCC(...
1    C/C(Cc1ccccc1CCC(=O)O)=C(\C)c1nc(C(=O)NCCc2ccc...
2      C/N=C(/NC#N)Nc1cccc(/C(=C\CCCC(=O)O)c2cccnc2)c1
3                     C=C(C)C(=O)O[Sn](CCCC)(CCCC)CCCC
4    C=C(NC(=O)c1ccc(/C(=C\CCCCC(=O)O)c2cccnc2)cc1)...
..                                                 ...
765  O=c1c(O)c(NCc2cccc(CCNS(=O)(=O)c3ccc(Cl)cc3)c2...
766  OC(O)(c1cncc(-c2sccc2-c2cc(Cl)ccc2OCc2ccccc2)c...
767                Oc1cc2c(cc1O)C(Cc1cc(I)cc(I)c1)NCC2
768                             Oc1ccc(Cl)cc1Cc1ccccc1
769                                             [Hg+2]

[770 rows x 1 columns]
     0  1  2  3  4  5  6  7  8  9  ...  2038  2039  2040  2041  2042  2043  \
0    0  0  0  0  0  0  0  0  1  0  ...     0     1     0     1     0     0   
1    0  0  0  0  0  0  0  0  1  0  ...     0     0     0     0     0     0   
2    0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
3   

C:\Users\mwhw3\Anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)



TEST BITVECT DFs CREATED

CALCULATING TANIMOTO SIMILARITIES...

NOW CALCULATING FOR THRESHOLD: 0.3

NOW CALCULATING FOR TARGET: TBXA2R
Current time:
05:54:51
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 0
Current time:
05:55:11
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 500
Current time:
05:55:31
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1000
Current time:
05:55:52
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1500
Current time:
05:56:12
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2000
Current time:
05:56:32
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2500
Current time:
05:56:52
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3000
Current time:
05:57:13
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3500
Current time:
05:57:33
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4000
Current time:
05:57:53
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4500
Current time:
05:58:13
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 5000
Current time:
05:58:3

C:\Users\mwhw3\Anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\indexes\base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
C:\Users\mwhw3\Anaconda3\envs\my-rdkit-env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)



TEST BITVECT DFs CREATED

CALCULATING TANIMOTO SIMILARITIES...

NOW CALCULATING FOR THRESHOLD: 0.3

NOW CALCULATING FOR TARGET: TEK
Current time:
06:05:25
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 0
Current time:
06:05:41
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 500
Current time:
06:05:56
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1000
Current time:
06:06:12
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 1500
Current time:
06:06:28
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2000
Current time:
06:06:43
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 2500
Current time:
06:06:59
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3000
Current time:
06:07:14
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 3500
Current time:
06:07:30
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4000
Current time:
06:07:45
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 4500
Current time:
06:08:01
NOW CALCULATING SIMILARITIES FOR PROCESS 1 INDEX 5000
Current time:
06:08:16
N